In [57]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_1_1.csv')
df_f = df[(df['ProductA가격'] != 0) & (df['ProductB가격'] != 0)].copy()
df_f['abs'] = abs(df_f['ProductA가격'] - df_f['ProductB가격'])
print(round(df_f.groupby('도시명')['abs'].mean().max(), 1))

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_1_2.csv')
df['Height_m'] = df['Height_cm'] / 100
df['bmi'] = round((df.Weight_kg) / (df.Height_m**2),1)
bmi_grade = []
for row in df['bmi']:
    if row >= 25:
        bmi_grade.append('비만')
    elif (row >= 23) & (row < 25):
        bmi_grade.append('과체중')
    elif (row >= 18.5) & (row < 23):
        bmi_grade.append('정상')
    elif row < 18.5:
       bmi_grade.append('저체중')
    else:
        bmi_grade.append('미분류')

df['bmi_grade'] = bmi_grade
print(df.bmi_grade.value_counts()['비만'] + df.bmi_grade.value_counts()['저체중'])

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_1_3.csv')
df['gen'] = df['products_made_domestic'] + df['products_made_international'] - df['products_sold_domestic'] - df['products_sold_international']
best = df.groupby(['factory', 'year'])['gen'].sum().groupby('year').idxmax()
result = sum(df.groupby(['factory', 'year'])['gen'].sum().loc[best])
print(result)

16333.3
74
9130


In [19]:
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_2_test.csv')

print(train.head(2))
print(test.head(2))

key = test['ID']
y_train = train['DBP']
x_train = train.drop(['level_0', 'ID', 'DBP'], axis=1)
x_test = test.drop(['level_0', 'ID', 'DBP'], axis=1)

co = pd.concat([x_train, x_test], axis=0)
co['Gender'] = co['Gender'].bfill()

from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

num = ['Age', 'BMI', 'FPG', 'Chol', 'Tri', 'HDL', 'LDL', 'ALT']
co = pd.get_dummies(co, columns=['Gender'])
scaler = StandardScaler()
co[num] = scaler.fit_transform(co[num])

x_train = co.iloc[:len(x_train),:]
x_test = co.iloc[len(x_train):,:]

x_t, x_va, y_t, y_va = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

model = RandomForestRegressor(
    n_estimators = 1000,
    max_depth = 10,
    min_samples_split = 4,
    min_samples_leaf = 3,
    random_state = 42
)

model.fit(x_t, y_t)
pred = model.predict(x_va)
print('MSE:', root_mean_squared_error(y_va, pred))

result = pd.DataFrame()
result['ID'] = key
result['pred'] = model.predict(x_test)
print(result.head(3))
#result.to_csv('result.csv', index=False)

   level_0    ID  Age Gender   BMI  DBP   FPG  Chol   Tri   HDL   LDL   ALT
0        0  2569   56    NaN  22.4   93  5.51  4.39  1.14  1.29  3.12  15.3
1        1  2215   33    NaN  22.1   75  5.51  3.32  0.75  1.15  1.95   9.7
   level_0    ID  Age Gender   BMI  DBP  FPG  Chol   Tri       HDL       LDL  \
0        0  2856   39    NaN  22.2   83  5.3  3.63  0.79  1.400000  1.740000   
1        1  3116   60    NaN  25.2   79  5.9  4.28  0.65  4.860753  4.860753   

    ALT  
0  13.0  
1  19.0  
MSE: 11.878128403912553
     ID       pred
0  2856  75.628854
1  3116  84.446941
2  1125  74.648337


In [59]:
from scipy.stats import kstest
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_3_1.csv')
print(round(df.work_hours.mean(), 3), round(df.work_hours.std(), 3))
st, p = kstest(df.work_hours, 'norm', args=(round(df.work_hours.mean(), 3),  round(df.work_hours.std(), 3)))
print(round(st, 3))
if p < 0.05:
    print('기각')
else:
    print('채택')

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_3_2.csv')
import statsmodels.formula.api as smf
print(df.head(3))
model = smf.ols('price ~ area + rooms + age', data=df).fit()
#print(model.summary())
print(model.params[model.params.index != 'Intercept'].idxmax())
print(len(model.params[(model.pvalues < 0.05) & (model.params.index != 'Intercept')]))

8.09 1.519
0.064
채택
           price        area  rooms  age
0  111842.374472  202.921570      2   11
1  109705.917098  162.004716      2    2
2  144253.316822  179.362140      5    5
rooms
3
